### Preprocessing, Filtering & Cleaning

##### This code section gets the incomming dataset and then detects the areas where NaN values are found. It then proceeds to fill these values with the means of their columns. However, the existence of NaN values in categorical columns is taken into account. In that case, the categorical NaN values are filled with the string "MISSING_VALUE".

In [16]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [17]:
# load test dataset
ds = pd.read_csv('wds/greek_weather_data.csv', delimiter="\t", low_memory=False)

In [18]:
ds.head()

,Unnamed: 0,STATION_REGION,STATION_NAME,YEAR,MONTH,DAY,MEAN_TEMP,HIGH_TEMP,HIGH_TEMP_TIME,LOW_TEMP,...,RAIN,AVG_WIND_SPEED,HIGHEST_WIND_SPEED,HIGHEST_WIND_SPEED_TIME,WIND_DIR,ELEVATION,LAT,LONG,LAT_DEG,LONG_DEG
0,0,Ήπειρος,Αγία Κυριακή Ιωαννίνων,2018,10,1,14.6,21.7,15:10:00,10.4,...,0.0,1.3,24.1,15:40:00,SSE,515.0,"39° 31' 27"" N","20° 52' 55"" E",39.524167,20.881944
1,1,Ήπειρος,Αγία Κυριακή Ιωαννίνων,2018,10,2,14.3,21.6,17:40:00,6.2,...,0.4,2.1,22.5,14:40:00,ENE,515.0,"39° 31' 27"" N","20° 52' 55"" E",39.524167,20.881944
2,2,Ήπειρος,Αγία Κυριακή Ιωαννίνων,2018,10,3,14.6,22.7,13:40:00,8.4,...,3.6,1.8,24.1,18:20:00,ESE,515.0,"39° 31' 27"" N","20° 52' 55"" E",39.524167,20.881944
3,3,Ήπειρος,Αγία Κυριακή Ιωαννίνων,2018,10,4,17.6,26.7,15:50:00,11.2,...,0.2,1.6,17.7,13:30:00,WNW,515.0,"39° 31' 27"" N","20° 52' 55"" E",39.524167,20.881944
4,4,Ήπειρος,Αγία Κυριακή Ιωαννίνων,2018,10,5,16.7,26.3,14:10:00,10.8,...,1.8,2.7,20.9,10:40:00,WNW,515.0,"39° 31' 27"" N","20° 52' 55"" E",39.524167,20.881944


In [19]:
#print the length of the dataset, as well as the columns where nan values exist
print("Dataset length:", len(ds))
print("NaN Values found:")
print(ds.isna().sum())

Dataset length: 921561
NaN Values found:
Unnamed: 0                      0
STATION_REGION                  0
STATION_NAME                    0
YEAR                            0
MONTH                           0
DAY                             0
MEAN_TEMP                    1118
HIGH_TEMP                    1179
HIGH_TEMP_TIME              35581
LOW_TEMP                     1117
LOW_TEMP_TIME               35537
HEAT_DEG_DAYS                2098
COOL_DEG_DAYS                2129
RAIN                         3651
AVG_WIND_SPEED               3400
HIGHEST_WIND_SPEED           3334
HIGHEST_WIND_SPEED_TIME     40459
WIND_DIR                        0
ELEVATION                  170187
LAT                        170187
LONG                       170187
LAT_DEG                    170187
LONG_DEG                   170187
dtype: int64


In [23]:
# print percentage of NaN values in each column
print("Percentage % of NaN values in each column:")
print(round(ds.isna().sum() * 100 / len(ds), 2))

Percentage % of NaN values in each column:
Unnamed: 0                  0.00
STATION_REGION              0.00
STATION_NAME                0.00
YEAR                        0.00
MONTH                       0.00
DAY                         0.00
MEAN_TEMP                   0.12
HIGH_TEMP                   0.13
HIGH_TEMP_TIME              3.86
LOW_TEMP                    0.12
LOW_TEMP_TIME               3.86
HEAT_DEG_DAYS               0.23
COOL_DEG_DAYS               0.23
RAIN                        0.40
AVG_WIND_SPEED              0.37
HIGHEST_WIND_SPEED          0.36
HIGHEST_WIND_SPEED_TIME     4.39
WIND_DIR                    0.00
ELEVATION                  18.47
LAT                        18.47
LONG                       18.47
LAT_DEG                    18.47
LONG_DEG                   18.47
dtype: float64


In [24]:
# RULES
# metadata info: rules that each column's values must have
def ruless(dss):
    print("Initial size before Rules application:",len(dss))
    
    # Temperatures must be between -100 and 55 degrees celsius
    dss = dss[dss['HIGH_TEMP'] < 55]
    dss = dss[dss['LOW_TEMP'] > -100]
    print("After Temperature Rule:", len(dss))
    
    # Rain must not be below 0
    dss = dss[dss['RAIN'] >= 0]
    print("After Rain Rule:", len(dss))
    
    # Highest wind speed must not be above 1000
    dss = dss[dss['HIGHEST_WIND_SPEED'] < 1000]
    print("After Wind Speed Rule:", len(dss))
    
    print("Final size after the Rules application:",len(dss))
    
    return dss

In [25]:
# metadata info: columns that will be parsed from outlier & wrong input detector
cols = ['MEAN_TEMP', 'HIGH_TEMP', 'LOW_TEMP', 'HEAT_DEG_DAYS', 'COOL_DEG_DAYS', 'RAIN', 'AVG_WIND_SPEED', 'HIGHEST_WIND_SPEED']

In [26]:
# search for outliers in the given columns
def outliered(dss):
    # a counter for the total outliers found
    totaloutliers = 0
    
    # begin parsing the given columns
    for cl in cols:
        # mean (average) calculation
        mean = dss[cl].mean()
        # standard deviation calculation
        std = dss[cl].std()
        
        # the cut-off threshold, which in this case is 3 times the standard deviation
        anomaly_cut_off = std * 3
        # calculate the lower limit. Below that, all values will be considered as outliers
        lower_limit  = mean - anomaly_cut_off 
        # calculate the upper limit. Above that, all values will be considered as outliers
        upper_limit = mean + anomaly_cut_off
        
        # get a list where 'True' values will indicate outliers above the upper limit
        outliers1 = dss[cl] > upper_limit
        # get a list where 'True' values will indicate outliers below the upper limit
        outliers2 = dss[cl] < lower_limit
        # merge the two lists. We now want all outliers. Simply, if we compare each value pairs between them with '!=',
        # we will get 'True' for the outliers (True != False equals True) and 'False' for correct values (False != False equals False)
        outliersfin = (outliers1 != outliers2)
        
        # convert the boolean list to a string list of Trues and Falses
        booleanDictionary = {True: 'True', False: 'False'}
        outliersfin = outliersfin.map(booleanDictionary)
        
        # add the outlier locations list to the main dataframe
        name = cl+"_OUTLIER_LOCATION"
        dss[name] = outliersfin
        
        # search for the outliers
        for it in outliersfin:
            if it == "True":
                totaloutliers = totaloutliers + 1
                
    print("Total outliers found:", totaloutliers)
    
    return dss

In [27]:
# function that cleans a given dataset, that is, it fills the empty numerical values with their columns' means,
# bypassing the empty categorical ones
def cleaner(dief):
    catcols = []
    numcols = []
    for c in dief.columns:
        #check if there are any strings in column
        if dief[c].map(type).eq(str).any(): 
            catcols.append(c)
        else:
            numcols.append(c)
            
    #create two DataFrames, one for each data type
    num = dief[numcols]
    cat = pd.DataFrame(dief[catcols])
    
    # create a num imputer for numerical valuees
    numimputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    # we apply the numimputer only to numeric columns / values
    num = pd.DataFrame(numimputer.fit_transform(num), columns = num.columns)
    
    # create a cat imputer for categorical valuees
    catimputer = SimpleImputer(strategy='constant', fill_value="MISSING_VALUE")
    # we apply the catimputer only to categorical columns / values
    cat = pd.DataFrame(catimputer.fit_transform(cat), columns = cat.columns)
    
    #join the two dataframes back together
    newdief = pd.concat([cat, num], axis = 1)
    
    # return the cleaned dataset
    return newdief

In [31]:
# apply the rules to the dataset
ds = ruless(ds)

Initial size before Rules application: 913565
After Temperature Rule: 913565
After Rain Rule: 913565
After Wind Speed Rule: 913565
Final size after the Rules application: 913565


In [30]:
# detect outliers in the dataset
ds = outliered(ds)

Total outliers found: 52972


In [12]:
# see the new dataframe with outlier locations columns
ds.head()

,Unnamed: 0,STATION_REGION,STATION_NAME,YEAR,MONTH,DAY,MEAN_TEMP,HIGH_TEMP,HIGH_TEMP_TIME,LOW_TEMP,...,LAT_DEG,LONG_DEG,MEAN_TEMP_OUTLIER_LOCATION,HIGH_TEMP_OUTLIER_LOCATION,LOW_TEMP_OUTLIER_LOCATION,HEAT_DEG_DAYS_OUTLIER_LOCATION,COOL_DEG_DAYS_OUTLIER_LOCATION,RAIN_OUTLIER_LOCATION,AVG_WIND_SPEED_OUTLIER_LOCATION,HIGHEST_WIND_SPEED_OUTLIER_LOCATION
0,0,Ήπειρος,Αγία Κυριακή Ιωαννίνων,2018,10,1,14.6,21.7,15:10:00,10.4,...,39.524167,20.881944,False,False,False,False,False,False,False,False
1,1,Ήπειρος,Αγία Κυριακή Ιωαννίνων,2018,10,2,14.3,21.6,17:40:00,6.2,...,39.524167,20.881944,False,False,False,False,False,False,False,False
2,2,Ήπειρος,Αγία Κυριακή Ιωαννίνων,2018,10,3,14.6,22.7,13:40:00,8.4,...,39.524167,20.881944,False,False,False,False,False,False,False,False
3,3,Ήπειρος,Αγία Κυριακή Ιωαννίνων,2018,10,4,17.6,26.7,15:50:00,11.2,...,39.524167,20.881944,False,False,False,False,False,False,False,False
4,4,Ήπειρος,Αγία Κυριακή Ιωαννίνων,2018,10,5,16.7,26.3,14:10:00,10.8,...,39.524167,20.881944,False,False,False,False,False,False,False,False


In [13]:
# replace nan values with the mean of each value's column. TAKES TIME
cleaned = cleaner(ds)

In [14]:
#validate that the length remains the same, as well as that the nan values have been eliminated
print("Dataset length after cleaning:", len(cleaned))
print("NaN Values found now:")
print(cleaned.isna().sum())
print("Dataset Now:")
print(cleaned)

Dataset length after cleaning: 913565
NaN Values found now:
STATION_REGION                         0
STATION_NAME                           0
DAY                                    0
HIGH_TEMP_TIME                         0
LOW_TEMP_TIME                          0
HIGHEST_WIND_SPEED_TIME                0
WIND_DIR                               0
LAT                                    0
LONG                                   0
MEAN_TEMP_OUTLIER_LOCATION             0
HIGH_TEMP_OUTLIER_LOCATION             0
LOW_TEMP_OUTLIER_LOCATION              0
HEAT_DEG_DAYS_OUTLIER_LOCATION         0
COOL_DEG_DAYS_OUTLIER_LOCATION         0
RAIN_OUTLIER_LOCATION                  0
AVG_WIND_SPEED_OUTLIER_LOCATION        0
HIGHEST_WIND_SPEED_OUTLIER_LOCATION    0
Unnamed: 0                             0
YEAR                                   0
MONTH                                  0
MEAN_TEMP                              0
HIGH_TEMP                              0
LOW_TEMP                              

In [15]:
# Create a correlation matrix
correlations = cleaned.corr()
correlations.style.background_gradient()

,Unnamed: 0,YEAR,MONTH,MEAN_TEMP,HIGH_TEMP,LOW_TEMP,HEAT_DEG_DAYS,COOL_DEG_DAYS,RAIN,AVG_WIND_SPEED,HIGHEST_WIND_SPEED,ELEVATION,LAT_DEG,LONG_DEG
Unnamed: 0,1.000000,0.054626,0.000419,0.072132,0.042258,0.094003,0.052919,0.064198,-0.043038,0.089704,0.099714,-0.131389,-0.192759,0.043670
YEAR,0.054626,1.000000,-0.035579,0.033950,0.033261,0.037534,0.291131,0.276703,-0.009553,-0.015200,-0.008063,-0.032031,-0.038081,0.042056
MONTH,0.000419,-0.035579,1.000000,0.201956,0.189932,0.212835,-0.012988,0.026296,-0.005326,-0.036313,-0.076310,-0.001557,-0.003348,0.004702
MEAN_TEMP,0.072132,0.033950,0.201956,1.000000,0.964415,0.958317,-0.098247,0.014266,-0.160209,0.054468,0.053104,-0.300774,-0.192082,0.122284
HIGH_TEMP,0.042258,0.033261,0.189932,0.964415,1.000000,0.862780,-0.103309,-0.023098,-0.192989,-0.066434,-0.046035,-0.266802,-0.125927,0.064788
LOW_TEMP,0.094003,0.037534,0.212835,0.958317,0.862780,1.000000,-0.083356,0.052428,-0.114642,0.158382,0.129301,-0.298388,-0.237355,0.159455
HEAT_DEG_DAYS,0.052919,0.291131,-0.012988,-0.098247,-0.103309,-0.083356,1.000000,0.929225,0.013755,0.016083,-0.007141,0.036779,0.000106,0.038539
COOL_DEG_DAYS,0.064198,0.276703,0.026296,0.014266,-0.023098,0.052428,0.929225,1.000000,0.024300,0.055446,0.018077,-0.019287,-0.049426,0.069719
RAIN,-0.043038,-0.009553,-0.005326,-0.160209,-0.192989,-0.114642,0.013755,0.024300,1.000000,0.027047,0.153842,0.059763,0.029137,-0.057063
AVG_WIND_SPEED,0.089704,-0.015200,-0.036313,0.054468,-0.066434,0.158382,0.016083,0.055446,0.027047,1.000000,0.808277,-0.023208,-0.191482,0.210118


In [ ]:
# save the new correlations dataset
correlations.to_csv("correlations.csv")